# Reshaping & Flattening
- **Reshaping**: Change array dimensions without altering data  
  `(4,) → (2,2)`  
- **Flattening**: Convert to 1D array  
  `(2,2) → (4,)`  

Key methods:  
- `.reshape()`: General reshaping  
- `.ravel()`: Fast flattening (view)  
- `.flatten()`: Safe flattening (copy)  

In [1]:
import numpy as np

# 1D to 2D
arr = np.arange(1, 7)  # [1,2,3,4,5,6]
matrix = arr.reshape(2, 3)
print("2x3 matrix:\n", matrix)
# [[1,2,3],
#  [4,5,6]]

# 1D to 3D
cube = arr.reshape(2, 3, 1)  # 2 layers, 3 rows, 1 column
print("\n3D cube shape:", cube.shape)  # (2,3,1)

# Automatic dimension with -1
auto_row = arr.reshape(2, -1)  # -1 = calculate cols (3)
auto_col = arr.reshape(-1, 2)  # -1 = calculate rows (3)
print("\nAuto row:\n", auto_row)
print("\nAuto col:\n", auto_col)

# Error: incompatible shape
try:
    arr.reshape(4, 4)  # 16 elements needed, only 6 exist
except ValueError as e:
    print("\nError:", e)

2x3 matrix:
 [[1 2 3]
 [4 5 6]]

3D cube shape: (2, 3, 1)

Auto row:
 [[1 2 3]
 [4 5 6]]

Auto col:
 [[1 2]
 [3 4]
 [5 6]]

Error: cannot reshape array of size 6 into shape (4,4)


### Flattening Options
| Method       | Returns | Memory | Speed  | Order       |
|--------------|---------|--------|--------|-------------|
| `.ravel()`   | View*   | Low    | Fast   | 'C' (row)   |
| `.flatten()` | Copy    | High   | Slow   | 'C' (row)   |
| `.reshape(-1)`| View*  | Low    | Fast   | 'C' (row)   |

*\* Returns view if possible, copy if not*

In [2]:
matrix = np.array([[1,2], [3,4]])

# ravel() (view when possible)
flat_ravel = matrix.ravel()
flat_ravel[0] = 99
print("Original after ravel mod:\n", matrix)  # Modified

# flatten() (always copy)
matrix = np.array([[1,2], [3,4]])  # Reset
flat_flatten = matrix.flatten()
flat_flatten[0] = 99
print("\nOriginal after flatten mod:\n", matrix)  # Unchanged

# reshape(-1) (view when possible)
flat_reshape = matrix.reshape(-1)
flat_reshape[1] = 88
print("\nOriginal after reshape mod:\n", matrix)  # Modified

Original after ravel mod:
 [[99  2]
 [ 3  4]]

Original after flatten mod:
 [[1 2]
 [3 4]]

Original after reshape mod:
 [[ 1 88]
 [ 3  4]]


## Reshaping Order: 'C' vs 'F'
- **'C' order**: Row-major (default)  
  `[[1,2,3], [4,5,6]] → [1,2,3,4,5,6]`  
- **'F' order**: Column-major  
  `[[1,2,3], [4,5,6]] → [1,4,2,5,3,6]`  

Controls how data is read/written during reshaping

In [3]:
matrix = np.array([[1,2], [3,4], [5,6]])

# C-order (row-wise) flattening
print("C-order:", matrix.ravel(order='C'))  # [1,2,3,4,5,6]

# F-order (column-wise) flattening
print("F-order:", matrix.ravel(order='F'))  # [1,3,5,2,4,6]

# Fortran-order reshaping
fortran_matrix = matrix.reshape(2, 3, order='F')
print("\nF-order reshaped:\n", fortran_matrix)
# [[1,5,4],
#  [3,2,6]]  (data filled column-wise)


C-order: [1 2 3 4 5 6]
F-order: [1 3 5 2 4 6]

F-order reshaped:
 [[1 5 4]
 [3 2 6]]


### Additional Methods
1. **`np.newaxis`**: Add new dimension  
   `(3,) → (3,1)` or `(1,3)`  
2. **`.transpose()`/.T**: Swap dimensions  
3. **`.squeeze()`**: Remove length-1 dimensions  
4. **`np.expand_dims()`**: Explicit dimension addition  

In [5]:
# np.newaxis
vector = np.array([1,2,3])
row_vec = vector[np.newaxis, :]  # 1x3
col_vec = vector[:, np.newaxis]  # 3x1
print("Row vector shape:", row_vec.shape)  # (1,3)
print("Column vector:\n", col_vec)
# [[1],
#  [2],
#  [3]]

# Transpose
matrix = np.arange(6).reshape(2,3)
print("\nOriginal:\n", matrix)
print("Transposed:\n", matrix.T)  # Swap rows/cols

# Squeeze
tensor = np.ones((1,3,1,4))
squeezed = tensor.squeeze()
print("\nOriginal shape:", tensor.shape)  # (1,3,1,4)
print("Squeezed shape:", squeezed.shape)  # (3,4)

# Expand dims
expanded = np.expand_dims(vector, axis=1)
print("\nExpanded shape:", expanded.shape)  # (3,1)

Row vector shape: (1, 3)
Column vector:
 [[1]
 [2]
 [3]]

Original:
 [[0 1 2]
 [3 4 5]]
Transposed:
 [[0 3]
 [1 4]
 [2 5]]

Original shape: (1, 3, 1, 4)
Squeezed shape: (3, 4)

Expanded shape: (3, 1)


### When to Use Reshaping
1. **ML Input Preparation**:  
   - Images: `(H,W,Channels)` → `(H*W, Channels)` for classic ML  
   - Sequences: `(samples, timesteps, features)` for RNNs  
2. **Broadcasting**:  
   Add dimensions for math operations: `vector[:, np.newaxis] * matrix`  
3. **Data Augmentation**:  
   Reshape batches for tensor operations  

In [7]:
# Image processing example
images = np.random.randint(0, 256, (100, 28, 28))  # 100 grayscale images

# Flatten for ML model
flattened = images.reshape(100, -1)  # (100, 784)

# Prepare for CNN (add channel dimension)
cnn_ready = images[:, :, :, np.newaxis]  # (100, 28, 28, 1)

# Time series for RNN
time_data = np.random.rand(365, 10)  # 365 days, 10 features
window_size = 7
sequences = np.array([time_data[i:i+window_size] for i in range(365-window_size)])
print("\nSequences shape:", sequences.shape)  # (358, 7, 10)


Sequences shape: (358, 7, 10)
